In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

## **Load dataset**

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape, y_train.shape

((60000, 28, 28), (60000,))

In [4]:
X_train = X_train.reshape(-1,784).astype("float32") / 255.0
X_test = X_test.reshape(-1,784).astype("float32") / 255.0
X_train.shape, X_test.shape

((60000, 784), (10000, 784))

## **Sequential API**
very convenient but not very flexible: can only map one input with one output

In [6]:
model = keras.Sequential(
    [
        keras.Input(shape=(28*28)),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(10),
    ]
)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    metrics=["accuracy"],
)

model.fit(X_train, y_train, batch_size=32, epochs=5, verbose=2)
model.evaluate(X_test, y_test, batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 27s - loss: 0.2915 - accuracy: 0.9190 - 27s/epoch - 14ms/step
Epoch 2/5
1875/1875 - 24s - loss: 0.1861 - accuracy: 0.9504 - 24s/epoch - 13ms/step
Epoch 3/5
1875/1875 - 27s - loss: 0.1610 - accuracy: 0.9582 - 27s/epoch - 14ms/step
Epoch 4/5
1875/1875 - 29s - loss: 0.1461 - accuracy: 0.9640 - 29s/epoch - 15ms/step
Epoch 5/5
1875/1875 - 27s - loss: 0.1363 - accuracy: 0.9671 - 27s/epoch - 14ms/step
313/313 - 2s - loss: 0.1926 - accuracy: 0.9590 - 2s/epoch - 5ms/step


[0.1926436573266983, 0.9589999914169312]

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 10)                2570      
                                                                 
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


In [8]:
import pickle
pickle.dump(model, open('model.pickle', 'wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-04-26 23:42:33         2443
metadata.json                                  2023-04-26 23:42:33           64
variables.h5                                   2023-04-26 23:42:33      6452440


In [15]:
model = keras.Sequential()
model.add(keras.Input(shape=(28*28)))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu', name='second_layer'))
model.add(layers.Dense(10))
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 512)               401920    
                                                                 
 dense_17 (Dense)            (None, 256)               131328    
                                                                 
 dense_18 (Dense)            (None, 10)                2570      
                                                                 
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________
None


**Debugging**

In [17]:
model = keras.Model(
    inputs=model.inputs,
    outputs=[model.layers[-2].output],
    # outputs=[model.get_layer('second_layer').output],
)
feature = model.predict(X_train)
feature.shape

1875/1875 [==============================] - 7s 4ms/step


(60000, 256)

In [ ]:
model = keras.Model(
    inputs=model.inputs,
    outputs=[layer.output for layer in model.layers]
)
features = model.predict(X_train)
for feature in features:
    print(feature.shape)

## **Functional API**
more flexible: handle multiple inputs and multiple outputs

In [13]:
inputs = keras.Input(shape=(28*28))
x = layers.Dense(512, activation='relu', name='first_layer')(inputs)
x = layers.Dense(256, activation='relu', name='second_layer')(x)
outputs = layers.Dense(10, activation='softmax', name='output_layer')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    metrics=["accuracy"],
)

model.fit(X_train, y_train, batch_size=32, epochs=5, verbose=2)
model.evaluate(X_test, y_test, batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 28s - loss: 0.3013 - accuracy: 0.9167 - 28s/epoch - 15ms/step
Epoch 2/5
1875/1875 - 27s - loss: 0.2035 - accuracy: 0.9457 - 27s/epoch - 14ms/step
Epoch 3/5
1875/1875 - 25s - loss: 0.1627 - accuracy: 0.9580 - 25s/epoch - 13ms/step
Epoch 4/5
1875/1875 - 28s - loss: 0.1471 - accuracy: 0.9624 - 28s/epoch - 15ms/step
Epoch 5/5
1875/1875 - 25s - loss: 0.1344 - accuracy: 0.9664 - 25s/epoch - 13ms/step
313/313 - 1s - loss: 0.1737 - accuracy: 0.9614 - 1s/epoch - 5ms/step


[0.17373208701610565, 0.9613999724388123]

In [ ]:
pickle.dump(model, open('model.pickle', 'wb'))